In [127]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision
import os 
from torchvision.transforms import ToTensor
from torchvision import datasets, transforms

In [128]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [129]:
train_dataset_path = 'Data/train'
test_dataset_path = 'Data/test'

In [130]:
train_dataset =  datasets.ImageFolder(root = train_dataset_path , transform = ToTensor())
test_dataset = datasets.ImageFolder(root = test_dataset_path, transform = ToTensor()) 

In [131]:
batch_size = 2000

# Create data loaders.
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


Shape of X [N, C, H, W]: torch.Size([2000, 3, 48, 48])
Shape of y: torch.Size([2000]) torch.int64


In [132]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)
model.dropout = nn.Dropout(0.5)
model.fc = nn.Linear(1024,7)
model.eval()
model.to(device)


Using cache found in C:\Users\henta/.cache\torch\hub\pytorch_vision_v0.10.0


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [133]:
# Define the device (CPU or GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Optimizing the model parameters
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
model.to(device)


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [134]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [135]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return 100*correct

In [136]:
test_accuracies = []
# Training and testing the model
epochs = 30
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss = train(train_dataloader, model, loss_fn, optimizer)
    acc = test(test_dataloader, model, loss_fn,)
    test_accuracies.append(acc)
print("Done!")
print(test_accuracies)    

Epoch 1
-------------------------------
loss: 2.327967  [    0/28709]
Test Error: 
 Accuracy: 22.9%, Avg loss: 1.969453 

Epoch 2
-------------------------------
loss: 1.961964  [    0/28709]
Test Error: 
 Accuracy: 23.7%, Avg loss: 1.913027 

Epoch 3
-------------------------------
loss: 1.891607  [    0/28709]
Test Error: 
 Accuracy: 24.6%, Avg loss: 1.883157 

Epoch 4
-------------------------------
loss: 1.865529  [    0/28709]
Test Error: 
 Accuracy: 25.4%, Avg loss: 1.856018 

Epoch 5
-------------------------------
loss: 1.859274  [    0/28709]
Test Error: 
 Accuracy: 26.0%, Avg loss: 1.833421 

Epoch 6
-------------------------------
loss: 1.820585  [    0/28709]
Test Error: 
 Accuracy: 26.1%, Avg loss: 1.819690 

Epoch 7
-------------------------------
loss: 1.911018  [    0/28709]
Test Error: 
 Accuracy: 26.7%, Avg loss: 1.804050 

Epoch 8
-------------------------------
loss: 1.790247  [    0/28709]
Test Error: 
 Accuracy: 27.5%, Avg loss: 1.793695 

Epoch 9
----------------

In [137]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [138]:
model = GoogLeNet()
model.load_state_dict(torch.load("model.pth"))
model.to(device)

C:\Users\henta\AppData\Local\Temp\ipykernel_31488\3433124577.py:70: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn('The default weight initialization of GoogleNet will be changed in future releases of '


RuntimeError: Error(s) in loading state_dict for GoogLeNet:
	Missing key(s) in state_dict: "aux1.conv.conv.weight", "aux1.conv.bn.weight", "aux1.conv.bn.bias", "aux1.conv.bn.running_mean", "aux1.conv.bn.running_var", "aux1.fc1.weight", "aux1.fc1.bias", "aux1.fc2.weight", "aux1.fc2.bias", "aux2.conv.conv.weight", "aux2.conv.bn.weight", "aux2.conv.bn.bias", "aux2.conv.bn.running_mean", "aux2.conv.bn.running_var", "aux2.fc1.weight", "aux2.fc1.bias", "aux2.fc2.weight", "aux2.fc2.bias". 
	size mismatch for fc.weight: copying a param with shape torch.Size([7, 1024]) from checkpoint, the shape in current model is torch.Size([1000, 1024]).
	size mismatch for fc.bias: copying a param with shape torch.Size([7]) from checkpoint, the shape in current model is torch.Size([1000]).